In [ ]:
### Use the Bayesian clasifer to detect the ground in a greenhouse
### @Dr. Zhaozheng Yin, Stony Brook University, Fall 2021
### housekeeping
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
### File directory
datadir = 'data-greenhouse\Tunnel-'
trainingImages = [1]
testingImages = [2, 3]
### Parameters
nDim = 8 #number of bins for the color likelihood distribution. Will also be trying 16 and 32.

In [ ]:
#Set Bins
def setBins(nDim): #Method to setup bins depending on nDim
    switcher = {
        8: [0, 32, 64, 96, 128, 160, 192, 224],
        16: [0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240],
        32: [0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 
            152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248]
    }
    return switcher.get(nDim)

In [ ]:
### Training process
Pr_x_given_y_equalsTo_1 = np.zeros((nDim,nDim,nDim)) #likelihood for the ground class
Pr_x_given_y_equalsTo_0 = np.zeros((nDim,nDim,nDim)) #likelihood for the non-ground class
N_GroundPixels = 0 #Pr_y_equalsTo_1 = N_GroundPixels/N_totalPixels
for iFile in trainingImages:
    ### Load the training image and labeled image regions
    print(datadir + '0' + str(iFile) + '.jpg')
    origIm = cv2.imread(datadir + '0' + str(iFile) + '.jpg')   
    labels = cv2.imread(datadir + '0' + str(iFile) + '-label.png', cv2.IMREAD_GRAYSCALE) #label=1 representing the ground class
    labels = cv2.threshold(labels, 127, 1, cv2.THRESH_BINARY)[1]
    
    ### Visualization input image and its labels
    nrows, ncols = origIm.shape[0], origIm.shape[1]
    showIm = origIm.copy() 
    showIm[labels==1] = 255;
    
    ### Be sure to convert the color space of the image from BGR (Opencv) to RGB (Matplotlib) before you show a color image read from OpenCV
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(origIm, cv2.COLOR_BGR2RGB))
    plt.title('Training image')
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(labels, 'gray')
    plt.title('GT')
    plt.axis("off")
    
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(showIm, cv2.COLOR_BGR2RGB))
    plt.title('GT overlyed on the training image')
    plt.axis("off")

    plt.show()
      
    ### Prior-related and likelihood-related codes (combining together):
    #Prior = P(y).
    #Likelihood = P(x|y)
    for h in range(nrows):
        for w in range(ncols):
            r = origIm[h, w, 0]
            g = origIm[h, w, 1]
            b = origIm[h, w, 2]
            temp = np.array([r, g, b])
            bins = setBins(nDim)
            index = np.digitize(temp, bins) #Bin the RGB.
            y = labels[h, w]
            if(y == 1):
                N_GroundPixels += 1 #This is a ground pixel
                Pr_x_given_y_equalsTo_1[index[0] - 1, index[1] - 1, index[2] - 1] += 1 #Mark the RGB in the np array
            else:
                Pr_x_given_y_equalsTo_0[index[0] - 1, index[1] - 1, index[2] - 1] += 1 #Mark the RGB in the np array
### Some other codes such as normalizing the likelihood/prior and computing Pr_y_equalsTo_0:
    Pr_y_equalsTo_1 = N_GroundPixels / (nrows * ncols)
    Pr_y_equalsTo_0 = 1 - Pr_y_equalsTo_1
    Pr_x_given_y_equalsTo_1 = Pr_x_given_y_equalsTo_1/N_GroundPixels
    Pr_x_given_y_equalsTo_0 = Pr_x_given_y_equalsTo_0/((nrows * ncols) - N_GroundPixels)

In [ ]:
### Testing
truePositives = 0;
falsePositives = 0;
falseNegatives = 0;
for iFile in testingImages:  
    ### Load the testing image and ground truth regions  
    origIm = cv2.imread(datadir + '0' + str(iFile) + '.jpg')
    gtMask = cv2.imread(datadir + '0' + str(iFile) + '-label.png', cv2.IMREAD_GRAYSCALE)
    gtMask = cv2.threshold(gtMask, 127, 1, cv2.THRESH_BINARY)[1]
    
    nrows, ncols = origIm.shape[0], origIm.shape[1] 
    
    ### Define the posteriors
    Pr_y_equalsTo_1_given_x = np.zeros((nrows,ncols));   
    Pr_y_equalsTo_0_given_x = np.zeros((nrows,ncols));

    ### Define the final result
    detectedMask = np.zeros((nrows, ncols))
    
    ### Codes to infer the posterior and obtain final classification result detectedMask:
    
    #Posterior: P(y|x)
    #Final Result: detectedMask
    for h in range(nrows):
        for w in range(ncols):
            r = origIm[h, w, 0]
            g = origIm[h, w, 1]
            b = origIm[h, w, 2]
            temp = np.array([r, g, b]) #Bin the RGB.
            bins = setBins(nDim)
            index = np.digitize(temp, bins) 
            Pr_y_equalsTo_1_given_x[h, w] = Pr_x_given_y_equalsTo_1[index[0] - 1, index[1] - 1, index[2] - 1] * Pr_y_equalsTo_1
            Pr_y_equalsTo_0_given_x[h, w] = Pr_x_given_y_equalsTo_0[index[0] - 1, index[1] - 1, index[2] - 1] * Pr_y_equalsTo_0
            if (Pr_y_equalsTo_1_given_x[h, w] > Pr_y_equalsTo_0_given_x[h, w]):
                detectedMask[h, w] = 1
            else:
                detectedMask[h, w] = 0
    
    ### Codes to calculate the TP, FP, FN:
    
    for h in range(nrows):
        for w in range(ncols):
            true_label = gtMask[h, w]
            if(gtMask[h, w] == detectedMask[h, w] and gtMask[h, w] == 1): #Correct Detection; True Positive
                truePositives += 1
            elif(gtMask[h, w] < detectedMask[h, w]): #gtMask = 0, detectedMask = 1; False Positive
                falsePositives += 1
            elif(gtMask[h, w] > detectedMask[h, w]): #gtMask = 1, detectedMask = 0; False Negative
                falseNegatives += 1
            else:
                pass #True Negative. We don't care.
    TP = truePositives / (nrows * ncols) #Normalize
    FP = falsePositives / (nrows * ncols) #Normalize
    FN = falseNegatives / (nrows * ncols) #Normalize
    
    ### Visualize the classification results
    showIm = origIm.copy()
    showIm[detectedMask==1] = 255;

    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(origIm, cv2.COLOR_BGR2RGB))
    plt.title('testing image')
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(detectedMask, 'gray')
    plt.title('detected mask')
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(showIm, cv2.COLOR_BGR2RGB))
    plt.title('detected mask overlyed on the testing image')
    plt.axis("off")
    plt.show()
    
    #Precision, Recall, F-Score:
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    fscore = (2 * precision * recall) / (precision + recall)
    print('Precision:' + str(precision))
    print('Recall:' + str(recall))
    print('F-Score:' + str(fscore))